## 4.1 전처리

### 4.1.2 전처리 과정 개요

1. 코퍼스 수집
2. 정제
3. 문장 단위 분절
4. 분절
5. 병렬 코퍼스 정렬(생략 가능)
6. 서브워드 분절

## 4.2 코퍼스 수집

1. 공개된 데이터 사용
2. 구매
-> 매우 한정적
3. 크롤링
-> 다양한 도메인(분야)의 코퍼스 모을 수 있음

> 주의 : 법적인 문제로 이어질 수 있음. 크롤링 허용여부는 robots.txt 확인

- selenium 패키지 사용해서 수행
- phantomJS or 크롬과 같은 헤드리스 브라우저 사용
- beautiful-soup 패키지 사용해 HTML 코드 파싱


### 4.2.1 단일 언어 코퍼스 수집

- 가장 손쉽게 구할 수 있는 종류의 코퍼스
- 위키피디아, 캐글 등에서 구할 수 있음


### 4.2.2 다중 언어 코퍼스 수집

- 자동 기계번역을 위한 병렬 코퍼스를 구하기란 단일 언어 코퍼스에 비해 상당히 어렵다
- 자막은 저작권이 있는 경우가 많아서 확인 필요

자막을 병렬 코퍼스로 사용할 경우 몇 가지 문제점이 있음
- 번역 품질의 저하로 인한 문제
- 'he' 나 'she' 와 같은 대명사가 사람 이름 등의 고유명사로 표현되는 문제
- 하지만 자막을 통해 매우 많은 양의 병렬 코퍼스를 얻을 수 있는 만큼 충분히 감수할 만한 문제점이라고 할 수 있음

## 4.3 정제

- 정제(normalization)는 텍스트를 사용하기에 앞서 필수적인 과정
- 원하는 업무와 문제에 따라, 또는 응용 분야에 따라 필요한 정제의 수준이나 깊이가 다를 수 있음
    - 음성 인식을 위한 언어 모델의 경우에는 사람의 음성을 그대로 받아 적어야 하므로 괄호 또는 별표와 같은 기호나 특수 문자들을 포함해서는 안됨
    - 전화번호나 이메일 주소, 신용카드 번호와 같은 개인정보나 민감한 정보들은 제거하거나 변조해서 모델링해야 할 수도 있음



### 4.3.1 전각 문자 제거

- 전각 문자 : 일반 영문자의 두배의 폭을 갖는 문자
- 대부분의 중국어와 일본어 문서, 그리고 일부 한국어 문서의 숫자, 영자, 기호가 전각 문자일때가 있음
- 이 경우, 일반적으로 사용되는 반각 문자로 변환해주는 작업이 필요

```
! " # $ % & ' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > ? @ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ \ ] ^ _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z { | } ~
```


### 4.3.2 대소문자 통일

- 일부 영어 코퍼스에서는 약자 등에서의 대소문자 표현이 통일되지 않을 때가 있음

    - New York City -> NYC, nyc, N.Y.C., N.Y.C

- 이러한 다양한 표현의 일원화는 하나의 의미를 지니는 여러 단어를 하나의 형태로 통일해 희소성(sparsity)을 줄이는 효과를 기대할 수 있음
- 딥러닝 시대에 접어들어 단어 임베딩을 통한 효율적인 표현이 가능해지면서 대소문자 통일같은 문제를 해결할 필요성이 줄어들었음


### 4.3.3 정규 표현식을 사용한 정제

- 크롤링을 통해 얻어낸 다량의 코퍼스는 보통 특수 문자, 기호 등에 의해 노이즈가 섞일 때가 많음
- 웹사이트의 성격에 따라 일정한 패턴을 지니는 경우도 많음
- 이러한 노이즈들을 효율적으로 감지하고 없애려면 인덱스의 사용은 필수

**| [ ] 사용 |**

```
'2 or 3 or 4 or 5 or c or d or e'

== [2345cde]

== (2|3|4|5|c|d|e)
```

**| - 사용 |**

- '-' 를 사용하여 연속된 숫자 또는 알파벳 등을 표현할 수 있음

```
'2 or 3 or 4 or 5 or c or d or e'

== [2-5c-e]
```

**| [^] 사용 |**

- Not 을 기호 '^' 를 써서 표현할 수 있음

```
2부터 5까지, 그리고 c부터 e까지를 제외한 한 글자를 의미

== [^2-5c-e]
```

**| ( ) 사용 |**

- 괄호를 이용해 그룹을 만들 수 있음

```
(x)(yz)
```

**| | 사용 |**

- '또는' 에 해당하는 or 을 의미

```
(x|y)
```

**| ?, *, + 사용 |**

- 앞의 수식하는 부분이 나타나지 않거나 한 번만 나타날 때는 '?'를 사용

```
x?
```

- 앞의 수식하는 부분이 한 번 이상 나타날 경우 '+'를 사용

```
x+
```

- 앞의 수식하는 부분이 나타나지 않거나 여러 번 나타날 경우 '*'를 사용

```
x*
```

**| {n}, {n,}, {n,m} 사용|**

- 정확하게 반복 횟수의 범위를 알고 있다면 사용하면 좋음

- 정확히 n번 반복하는 경우

```
x{n}
```

- n번 이상 반복하는 경우

```
x{n,}
```

- n번에서 m번 사이를 반복하는 경우

```
x{n,m}
```

**| . 사용 |**

- '.'은 어떤 글자도 포함
- 유의해서 사용해야 함

```
.
```

**| ^와 $ 사용 |**

- '[' 과 ']' 내에 포함되지 않은 '^'은 라인의 시작을 의미하며 '$'은 라인의 종료를 의미

```
^x$
```

**| 지정 문자 사용 |**

- \s : 공백 문자(white space)
- \S : 공백 문자를 제외한 모든 문자
- \w : alphanumeric(알파벳 + 숫자) + '_'([A-Za-z0-9_]와 같음)
- \W : non-alphanumeric 문자 및 '_' 제외([^A-Za-z0-9_]와 같음)
- \d : 숫자([0-9]와 같음)
- \D : 숫자를 제외한 모든 문자([^0-9]와 같음)

**| 예제 |**

- 문서의 마지막 줄에 전화번호 등의 개인 정보가 포함된 문서를 데이터셋으로 사용하려 함
- 이때 해당 개인 정보를 제외하고 사용하고자 함

```
Hello Ki, I would like to introduce regular expression in this setion.
~~
Thank you!
Sincerely,
Ki: +82-10-1234-5678
```

- 무턱대고 마지막 줄을 지우자니, 마지막 줄에 전화번호 정보가 없는 경우도 많기 때문에 선택적으로 지워야 함
- 데이터를 훑어가며 살펴보니 마지막 줄은 다음과 같은 규칙을 따르는 듯함

    - 이름이 전화번호 앞에 나올 수도 있다.
    - 이름 뒤에는 콜론(:)이 나올 수도 있다.
    - 콜론 앞/뒤로는 (탭을 포함한) 공백이 다수 존재할 수도 있다.
    - 전화번호는 국가번호를 포함할 수도 있다.
    - 국가번호는 최대 3자리이다.
    - 국가번호의 앞에는 '+'가 붙을 수도 있다.
    - 전화번호 사이에 '-'가 들어갈 수도 있다.
    - 전화번호는 빈칸 없이 표현된다.
    - 전화번호 맨 앞과 지역번호(또는 010)의 다음에는 괄호가 들어갈 수도 있다.
    - 괄호는 한쪽만 나올 수도 있다.
    - 지역번호 자리의 맨 처음에 나오는 0은 빠질 수도 있다. 즉, 2자리가 될 수도 있다.
    - 지역번호 다음 번호 그룹은 3에서 4자리 숫자이다.
    - 마지막은 항상 4자리 숫자이다.

- 위의 규칙을 따르는 정규 표현식은 다음과 같다.

```
([\w]+\s*:?\s*)?\(?\+?([0-9]{1,3})?\-?[0-9]{2,3}(\)|\-)?[0-9]{3,4}\-?[0-9]{4}
```


**| 파이썬에 정규표현식 사용 |**




In [1]:
import re

In [2]:
regex = r"([\w]+\s*:?\s*)?\(?\+?([0-9]{1,3})?\-?[0-9]{2,3}(\)|\-)?[0-9]{3,4}\-?[0-9]{4}"

In [3]:
x = "Ki: +82-10-9420-4104"
re.sub(regex, 'REMOVED', x)

'REMOVED'

In [4]:
x = 'CONTENT jiu 02)9420-4104'
re.sub(regex, 'REMOVED', x)

'CONTENT REMOVED'

**| 치환자 사용 |**

문제 : 알파벳(소문자) 사이에 있는 숫자를 제거하시오.

```
abcdefg
12345
ab12
a1bc2d
12ab
a1b
1a2
a1
1a
hijklmnop
```

In [5]:
x = '''
abcdefg
12345
ab12
a1bc2d
12ab
a1b
1a2
a1
1a
hijklmnop
'''

In [6]:
regex = r'([a-z])[0-9]+([a-z])'
to = r'\1\2'  # \1 : 첫번째괄호, \2 : 두번째괄호

In [7]:
y = '\n'.join([re.sub(regex, to, x_i) for x_i in x.split('\n')])

In [9]:
print(y)


abcdefg
12345
ab12
abcd
12ab
ab
1a2
a1
1a
hijklmnop



## 4.4 문장 단위 분절

- 보통 다루려는 문제들은 입력 단위가 문장 단위인 경우가 많음
- 여러 문장이 한 라인에 있거나, 한 문장이 여러 라인에 걸쳐 있는 경우에는 문장 단위 분절이 필요

### 4.4.1 문장 단위 분절 예제

```
자연어처리는 인공지능의 한 줄기 입니다. 시퀀스 투 시퀀스의 등장 이후로 딥러닝을 활용한 자연어처리는 새로운 전기를 맞이하게 되었습니다. 문장을 받아 단순히 수치로 나타내던 시절을 넘어, 원하는대로 문장을 만들어낼 수 있게 된 것입니다.
```

In [30]:
import sys, fileinput, re
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
f = open("text.txt", "w")
f.write("자연어처리는 인공지능의 한 줄기 입니다. 시퀀스 투 시퀀스의 등장 이후로 딥러닝을 활용한 자연어처리는 새로운 전기를 맞이하게 되었습니다. 문장을 받아 단순히 수치로 나타내던 시절을 넘어, 원하는대로 문장을 만들어낼 수 있게 된 것입니다.")
f.close()

In [34]:
if __name__ == '__main__':
    for line in fileinput.FileInput('text.txt'):
        if line.strip() != '':
            line = re.sub(r'([a-z])\.([A-Z])', r'\1. \2', line.strip())

            sentences = sent_tokenize(line.strip())

            for s in sentences:
                if s != '':
                    sys.stdout.write(s + '\n')

자연어처리는 인공지능의 한 줄기 입니다.
시퀀스 투 시퀀스의 등장 이후로 딥러닝을 활용한 자연어처리는 새로운 전기를 맞이하게 되었습니다.
문장을 받아 단순히 수치로 나타내던 시절을 넘어, 원하는대로 문장을 만들어낼 수 있게 된 것입니다.


### 4.4.2 문장 합치기 및 분절 예제

- 여러 라인에 걸쳐 한 문장이 들어 있는 경우

```
자연어처리는 인공지능의 한 줄기 입니다. 시퀀스 투 시퀀스의 등장 이후로\n
딥러닝을 활용한 자연어처리는 새로운 전기를 맞이하게 되었습니다. 문장을\n
받아 단순히 수치로 나타내던 시절을 넘어, 원하는대로 문장을 만들어낼 수 \n
있게 된 것입니다.
```

In [1]:
f = open("text2.txt", "w")
f.write("자연어처리는 인공지능의 한 줄기 입니다. 시퀀스 투 시퀀스의 등장 이후로\n")
f.write("딥러닝을 활용한 자연어처리는 새로운 전기를 맞이하게 되었습니다. 문장을\n")
f.write("받아 단순히 수치로 나타내던 시절을 넘어, 원하는대로 문장을 만들어낼 수 \n")
f.write("있게 된 것입니다.")
f.close()

In [4]:
import sys, fileinput
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
if __name__ == '__main__':
    buf = []

    for line in fileinput.FileInput('text2.txt'):
        if line.strip() != '':
            buf += [line.strip()]
            sentences = sent_tokenize(' '.join(buf))

            if len(sentences) > 1:
                buf = sentences[-1:]

                sys.stdout.write('\n'.join(sentences[:-1]) + '\n')

    sys.stdout.write(' '.join(buf) + '\n')

자연어처리는 인공지능의 한 줄기 입니다.
시퀀스 투 시퀀스의 등장 이후로 딥러닝을 활용한 자연어처리는 새로운 전기를 맞이하게 되었습니다.
문장을 받아 단순히 수치로 나타내던 시절을 넘어, 원하는대로 문장을 만들어낼 수 있게 된 것입니다.


## 4.5 분절

- 풀고자 하는 문제에 따라 형태소 분석 또는 단순한 분절(띄어쓰기)을 통해 정규화를 수행
- 한국어의 경우 띄어쓰기 관련 표준화 과정이 충분하지 않아 띄어쓰기가 중구난방인 경우가 많음
- 한국어는 띄어쓰기가 문장의 의미 해석에 큰 영향을 끼치지 않음
- 띄어쓰기가 되어 있더라도 제각각인 경우가 많아서 표준화된 띄어쓰기를 적용하는 과정이 필요
- 교착어로써 접사를 어근에서 분리해주는 역할도 하므로 희소성 문제를 해소하기도 함

- 영어의 경우 기본적으로 띄어쓰기가 있고 대부분의 경우 규칙을 매우 잘 따르고 있음

- 언어별 주요 프로그램
    - 한국어 : Mecab, KoNLPy
    - 일본어 : Mecab
    - 중국어 : Standard Parser, PKU Parser, Jieba

- 각 프로그램 성능의 차이를 만드는 것은 신조어나 보지 못한 고유명사를 처리하는 능력임
- 자신이 풀고자 하는 문제의 특징을 잘 파악하여 그에 알맞은 정책을 가진 프로그램을 선택하는 것이 중요

### 4.5.1 한국어 분절

**| Mecab |**

- 한국어 분절에 가장 많이 사용되는 프로그램
    ```
    $ sudo apt-get install curl
    $ bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
    ```

- 설치된 Mecab 은 KoNLPy 에서 불러 사용할 수 있음
    ```
    $ echo "안녕하세요, 반갑습니다!" | mecab
    ```


In [10]:
!sudo apt-get install curl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.16).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [11]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Installing automake (A dependency for mecab-ko)
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,628 kB]
Get:14 http://ppa.launchpad.net/deadsna

In [12]:
!echo "안녕하세요, 반갑습니다!" | mecab

안녕	NNG,행위,T,안녕,*,*,*,*
하	XSV,*,F,하,*,*,*,*
세요	EP+EF,*,F,세요,Inflect,EP,EF,시/EP/*+어요/EF/*
,	SC,*,*,*,*,*,*,*
반갑	VA,*,T,반갑,*,*,*,*
습니다	EF,*,F,습니다,*,*,*,*
!	SF,*,*,*,*,*,*,*
EOS


- 단순히 띄어쓰기만 적용하고 싶을 경우에는 -O wakati 옵션을 사용하여 실행하면 됨
    ```
    $ echo "안녕하세요, 반갑습니다!" | mecab -O wakati
    ```


In [13]:
!echo "안녕하세요, 반갑습니다!" | mecab -O wakati

안녕 하 세요 , 반갑 습니다 ! 


- 기본적으로 어근과 접사를 분리하는 역할을 수행하며, 나악 ㅏ잘못된 띄어쓰기에 대해서도 올바르게 교정해주는 역할
- 어근과 접사를 분리함으로써 어휘의 숫자를 효과적으로 줄여 희소성을 해소할 수 있음

**| KoNLPy |**

- 한국어 형태소 분식기들을 모아놓은 랩핑 라이브러리
- 자바로 구현되어 있어 C++ 로 구현된 Mecab 에 비해 속도면에서 불리
- 대용량의 코퍼스를 처리할 때에 불리
- 설치 및 사용이 쉽고 다양한 라이브러리들을 모아놓았다는 장점

### 4.5.2 영어 분절

**| Moses |**

- 띄어쓰기 자체에 대한 큰 정규화 이슈는 없음
- 쉼표(comma), 마침표(period), 인용부호(quotation) 등을 띄어 주어야 함

- NLTK(3.2.5)에서는 Moses 를 포함했으므로 분절 기능을 제공했음
    ```
    $ pip install nltk==3.2.5
    ```

- NLTK(3.2.5 버전)를 사용한 파이썬 스크립트 예제
    ```
    Natural language processing is one of biggest streams in artificial intelligence, and it becoms very popular after seq2seq2's invention.
    ```
    

In [14]:
!pip install nltk==3.2.5

In [20]:
import nltk
nltk.download('perluniprops')
nltk.download('nonbreaking_prefixes')

import sys, fileinput

from nltk.tokenize.moses import MosesTokenizer

[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


In [18]:
f = open('text3.txt', 'w')
f.write("Natural language processing is one of biggest streams in artificial intelligence, and it becoms very popular after seq2seq2's invention.")
f.close()

In [21]:
t = MosesTokenizer()

if __name__ == '__main__':
    for line in fileinput.FileInput('text3.txt'):
        if line.strip() != '':
            tokens = t.tokenize(line.strip(), escape=False)

            sys.stdout.write(' '.join(tokens) + '\n')
        else:
            sys.stdout.write('\n')

Natural language processing is one of biggest streams in artificial intelligence , and it becoms very popular after seq2seq2 's invention .


### 4.5.4 중국어 분절

**| 스탠포드 파서 |**

- 스탠포드 대학교에서 제작한 중국어 파서 프로그램
- 자바로 제작
- 참고사이트 : https://nlp.stanford.edu/software/lex-parser.shtml

**| JIEBA |**

- 성능상으로는 다른 형태소 분석기들과 큰 차이가 없음
- 파이썬으로 구현되었기에 실제 사용화를 위한 배치 시스템을 구현할 때 매우 쉽고 용이함
- 참고사이트 : https://github.com/fxsjy/jieba


## 4.6 병렬 코퍼스 정렬

- 대부분의 병렬 코퍼스들은 여러 문장 단위로 정렬됨
- 문서와 문서 단위의 맵핑은 있는 반면 문장 대 문장에 관한 정렬은 이루어져 있지 않음
- 이 경우 각각의 문장에 대해 정렬해주어야 하고 그 과정에서 일부 불필요한 문장들을 걸러내야 하고, 문장 간 정렬이 잘 맞지 않는 경우 정렬을 재정비하거나 아예 걸러내야 함

### 4.6.1 병렬 코퍼스 제작 프로세스 개요

1. 소스 언어(source language)와 타깃 언어(target language) 사이의 단어 사전을 준비합니다.
2. 만약 준비된 단어 사전이 없다면 다음 작업을 수행합니다. 만약 이미 단어 사전을 갖고 있다면 7번으로 건너뜁니다.
3. 각 언어에 대해서 코퍼스를 수집 및 정제합니다.
4. 각 언어에 대해서 단어 임베딩 벡터를 구합니다. 단어 임베딩 벡터에 대해서는 추후 다루겠습니다.
5. MUSE 를 통해 단어 레벨 번역기를 훈련합니다.
6. 훈련된 단어 레벨 번역기를 통해 두 언어 사이의 단어 사전을 생성합니다.
7. 만들어진 단어 사전을 넣어 Champollion 을 통해 기존에 수집된 다중 언어 코퍼스를 정렬합니다.
8. 각 언어에 대해서 단어 사전을 적용하기 위해 알맞은 수준의 분절을 수행합니다.
9. 각 언어에 대해서 정제를 수행합니다.
10. Champollion 을 사용하여 병렬 코퍼스를 생성합니다.

### 4.6.2 사전 생성

- 페이스북의 MUSE 는 병렬 코퍼스가 없는 상황에서 사전을 구축하는 방법과 코드를 제공함
- 각 단일 언어 코퍼스를 통해 구축한 언어별 단어 임베딩 벡터에 대해 다른 언어의 임베딩 벡터와 맵핑시켜 단어 간 번역을 수행할 수 있음
- 각 언어별 코퍼스가 많을수록, 임베딩 벡터가 많을수록 더욱 정확하게 수행됨
- MUSE 는 병렬 코퍼스가 없는 상황에서도 수행할 수 있기 때문에 비지도학습이라고 할 수 있음

실제로 MUSE 를 통해 비지도학습을 사용하여 결과물로 얻은 영한 단어 번역 사전의 일부

```
stories <> 이야기
stories <> 소설
contact <> 연락
contact <> 연락처
contact <> 접촉
green <> 녹색
green <> 초록색
green <> 빨간색
dark <> 어두운
dark <> 어둠
dark <> 짙
song <> 노래
song <> 곡
song <> 음악
salt <> 소금
```

- CTK 의 입력으로 사용되고 CTK 은 이 사전을 바탕으로 병렬 코퍼스의 문장 정렬을 수행
- '<>' 을 구분 문자로 사용하여 한 라인에 소스 언어의 단어와 타깃 언어의 단어를 표현

### 4.6.3 CTK 을 활용한 정렬

- CTK(Champollion Toolkit)은 이중 언어 코퍼스의 문장 정렬을 수행하는 오픈소스
- 펄(Perl)을 사용하여 구현되었음
- 기존에 구축된 단어 사전을 이용하거나, 앞에서와 같이 자동으로 구축한 단어 사전을 참고하여 CTK 은 문장 정렬을 수행함

- 여러 라인으로 구성된 언어별 문서에 대해 문장 정렬한 결과의 예제

```
omitted <=> 1
omitted <=> 2
omitted <=> 3
1 <=> 4
2 <=> 5
3 <=> 6
4, 5 <=> 7
6 <=> 8
7 <=> 9
8 <=> 10
9 <=> omitted
```

- 타깃 언어의 1, 2, 3번째 문장은 짝을 찾지 못하고 버려졌고, 솟 언어의 1, 2, 3번째 문장은 각각 타깃 언어의 4, 5, 6번째 문장과 맵핑된 것을 알 수 있음
- 소스 언어의 4, 5번째 두 문장은 타깃 언어의 7번 문장에 동시에 맵핑된 것을 알 수 있음
- 어떤 문장들은 버려지기도 하고, 일대일 맵핑이 이루어지기도 하며, 일대다, 다대일 맵핑이 이루어지기도 함

## 4.7 서브워드 분절

- BPE(byte pair encoding) 알고리즘을 통한 서브워드 단위 분절은 현재 필수 전처리 방법으로 자리잡았음
- 기본적으로 '단어는 의미를 가진 더 작은 서브워드들의 조합으로 이루어진다'는 가정하에 적용되는 알고리즘
- 영어나 한국어 역시 라틴어와 한자를 기반으로 형성된 언어이기에 많은 단어가 서브워드들로 구성됨
- 적절한 서브워드를 발견하여 해당 단위로 쪼개어주면 어휘 수를 줄일 수 있고 희소성을 효과적으로 줄일 수 있음
- 서브워드 단위 분절로 얻는 가장 대표적인 효과는 UNK(unknown) 토큰에 대한 효율적인 대처임
    - 신조어나 오타(typo)와 같은 UNK 에 대해 서브워드 단위나 문자(character) 단위로 쪼개줌으로써 기존 훈련 데이터에서 보았던 토큰들의 조합으로 만들어버릴 수 있음
    - UNK 자체를 없앰으로써 효율적으로 UNK 에 대처할 수 있고, 자연어 처리 알고리즘의 결과물 품질을 향상시킬 수 있음
- 다만, 한글과 영어만으로 훈련한 알고리즘 또는 모델에 '아랍어'와 같이 전혀 보지 못한 글자가 등장한다면 당연히 UNK 로 치환될 것임

### 4.7.1 예제

- BPE 를 적용하면 원래의 띄어쓰기 공백 외에도 BPE 의 적용으로 인한 공백이 추가됨
- 원래의 띄어쓰기와 BPE 로 인한 띄어쓰기를 서로 구분해야 함
- 특수문자 _ 를 사용하여 기존의 띄어쓰기 또는 단어의 시작을 나타냄
- 이를 통해 분절된 문장을 BPE 이전의 형태로 원상복구할 수 있음

- 한글 Mecab 에 의해 분절된 원문
    ```
    자연어 처리 는 인공지능 의 한 줄기 입니다 .
    시퀀스 투 시퀀스 의 등장 이후 로 딥 러닝 을 활용 한 자연어 처리 는 새로운 전기 를 맞이 하 게 되 었 습니다 .
    문장 을 받 아 단순히 수치 로 나타내 던 시절 을 넘 어 , 원 하 는 대로 문장 을 만들 어 낼 수 있게 된 것입니다 .
    이 에 따라 이전 까지 큰 변화 가 없 었 던 자연어 처리 분야 의 연구 는 폭발 적 으로 늘어나 기 시작 하 여 , 곧 기계번역 시스템 은 신경망 에 의해 정복 당하 였 습니다 .
    또한 , attention 기법 의 고도 화 로 전이학습 이 발전 하 면서 , QA 문제 도 사람 보다 정확 한 수준 이 되 었 습니다 .
    ```

- 한글 BPE 적용
    ```
    _자연 어 _처리 _는 _인공지능 _의 _한 _줄기 _입니다 _.
    _시퀀스 _투 _시퀀스 _의 _등장 _이후 _로 _딥 _러 닝 _을 _활용 _한 _자연 어 _처리 _는 _새로운 _전기 _를 _맞이 _하 _게 _되 _었 _습니다 _.
    _문장 _을 _받 _아 _단순히 _수치 _로 _나타내 _던 _시절 _을 _넘 _어 _, _원 _하 _는 _대로 _문장 _을 _만들 _어 _낼 _수 _있 _게 _된 _것 _입니다 _.
    _이 _에 _따라 _이전 _까지 _큰 _변화 _가 _없 _었 _던 _자연 어 _처리 _분야 _의 _연구 _는 _폭발 _적 _으로 _늘어나 _기 _시작 _하 _여 _, _곧 _기계 _번역 _시스템 _은 _신경 망 _에 _의해 _정복 _당하 _였 _습니다 _.
    _또한 _, _attention _기법 _의 _고도 _화 _로 _전이 _학습 _이 _발전 _하 _면서 _, _Q A _문제 _도 _사람 _보다 _정확 _한 _수준 _이 _되 _었 _습니다 _.
    ```

- 영어 NLTK 에 의해 분절된 원문
    ```
    Natural language processing is one of biggest streams in artificial intelligence , and it becomes very popular after seq2seq 's invention .
    However , in order to make a strong A.I . , there are still many challenges remain .
    I believe that we can breakthrough these barriers to get strong artificial intelligence .
    ```

- 영어 BPE 적용
    ```
    _Natural _language _processing _is _one _of _beggest _stream s _in _ artificial _intelligence _, _and _it _becomes _very _popular _after _se q 2 se q _'s _invention _.
    _However _, _in _order _to _make _a _strong _A. I _. _, _there _are _still _many _challenges _remain _.
    _I _believe _that _we _can _breakthrough _these _barriers _to _get _strong _artificial _intelligence _.
    ```

### 4.7.2 오픈 소스

- 구글의 SentencePiece 모듈이 속도는 빠르지만, 논문 원저자의 파이썬 코드는 수정이 쉬워 편의에 따라 사용하면 됨
    - Sennrich(원저자)의 깃허브 : https://github.com/rsennrich/subword-nmt
    - 본서의 저자가 수정한 버전 : https://github.com/kh-kim/subword-nmt
    - Google 의 SentencePiece 모듈 : https://github.com/google/sentencepiece

## 4.8 분절 복원

```
Natural language processing is one of biggest streams in artificial intelligence, and it becomes very popular after seq2seq's invention.
```

- 언어별 분절기 모듈(영어의 경우 NLTK)을 통해 분절을 수행하고, 기존 띄어쓰기와 새롭게 분절기에 의해 만들어진 공백을 구분하기 위해 특수 문자 '▁' 를 원래의 공백 위치에 삽입함
- 이 때, '▁' 문자는 '_' 와 다른 문자로 특수기호임

```
▁Natural ▁language ▁processing ▁is ▁one ▁of ▁biggest ▁streams ▁in ▁artificial ▁intelligence , ▁and ▁it ▁becomes ▁very ▁popular ▁after ▁seq2seq 's ▁invention .
```

- 여기에 서브워드 단위 분절을 수행하며 이전 과정까지의 공백과 서브워드 단위 분절로 인한 공백을 구분하기 위한 특수문자 '▁'를 삽입

```
▁▁Natural ▁▁language ▁▁processing ▁▁is ▁▁one ▁▁of ▁▁biggest ▁▁streams ▁▁in ▁▁artificial ▁▁intelligence ▁, ▁▁and ▁▁it ▁▁becomes ▁▁very ▁▁popular ▁▁after ▁▁se q 2 se q ▁'s ▁▁invention ▁.
```

- 이런 형태의 분절된 문장을 자연어 처리 모델에 훈련시키면 똑같은 형태로 분절된 문장을 생성해낼 것임
- 그럼 이런 문장을 분절 복원하여 사람이 읽기 좋은 형태로 만들어주어야 함

- 먼저 공백을 제거하자

```
▁▁Natural▁▁language▁▁processing▁▁is▁▁one▁▁of▁▁biggest▁▁streams▁▁in▁▁artificial▁▁intelligence▁,▁▁and▁▁it▁▁becomes▁▁very▁▁popular▁▁after▁▁seq2seq▁'s▁▁invention▁.
```

- 특수 문자 '▁' 가 2개가 동시에 있는 문자열 '▁▁' 을 공백으로 치환
- 이러면 한 개 짜리 '▁' 만 남음

```
Natural language processing is one of biggest streams in artificial intelligence▁, and it becomes very popular after seq2seq▁'s invention▁.
```

- 마지막 남은 특수 문자 '▁' 를 제거하면 문장 복원이 완성

```
Natural language processing is one of biggest streams in artificial intelligence, and it becomes very popular after seq2seq's invention.
```


### 4.8.1 분절 후처리

- 분절에 대한 복원을 수월하게 하기 위해, 분절 이후에는 특수 문자를 분절과정에서 새롭게 생긴 공백 다음에 삽입해야 함